## Load and Clean the Data

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

# Define the tickers and the date range
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-01-01'
end_date = '2025-01-31'

# Fetch the data
data = yf.download(tickers, start=start_date, end=end_date)

# Extract the 'Close' prices
close_prices = data['Close']

# Check for missing values
print(close_prices.isnull().sum())

# Handle missing values (e.g., forward fill)
close_prices.fillna(method='ffill', inplace=True)

# Ensure all columns have appropriate data types
print(close_prices.dtypes)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed


Ticker
BND     0
SPY     0
TSLA    0
dtype: int64
Ticker
BND     float64
SPY     float64
TSLA    float64
dtype: object


/tmp/ipykernel_94760/2884662836.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  close_prices.fillna(method='ffill', inplace=True)
/tmp/ipykernel_94760/2884662836.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_prices.fillna(method='ffill', inplace=True)
